# Implicit Gyroid structure
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/gyroid.ipynb)

In [1]:
import torch
from torchfem import Solid
from torchfem.materials import IsotropicElasticity3D
from torchfem.mesh import cube_hexa
from torchfem.sdfs import gyroid

import pyvista

pyvista.set_plot_theme("document")
torch.set_default_dtype(torch.float64)

In [2]:
# Material parameters
E = 1000.0
nu = 0.3

# Gyroid parameters
scale = 2.0
thickness = 0.1

# Mesh parameters
N = 51

# Homogenization parameters
disp = 0.1
rho_min = 0.01

## Create voxel mesh

In [3]:
# Create a mesh
nodes, elements = cube_hexa(51, 51, 51)

## Evaluate signed distance function

In [4]:
# Create unstructured mesh
elems = [item for element in elements for item in (len(element), *element)]
cell_types = len(elements) * [pyvista.CellType.HEXAHEDRON]
mesh = pyvista.UnstructuredGrid(elems, cell_types, nodes.tolist())

# Evaluate signed distance function
SDF = gyroid(scale * nodes)
mesh.point_data["SDF"] = SDF.numpy()

## Plot signed distance function

In [5]:
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh, cmap="coolwarm", show_edges=True, clim=[-1.0, 1.0])
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Plot shell gyroid 

In [6]:
# Contours
contours = mesh.contour([-thickness, thickness])

# Plot
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh.outline(), color="k")
pl.add_mesh(contours, color="gray")
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [7]:
# Elastic material model
material = IsotropicElasticity3D(E=E, nu=nu)

# Create model
model = Solid(nodes, elements, material)

# Set constraints
model.constraints[nodes[:, 0] == 0.0, 0] = True
model.constraints[nodes[:, 1] == 0.5, 1] = True
model.constraints[nodes[:, 2] == 0.5, 2] = True
model.constraints[nodes[:, 0] == 1.0, 0] = True
model.displacements[nodes[:, 0] == 1.0, 0] = 0.1

# Create nodal density field
mask = torch.abs(SDF) > thickness
rho_nodes = torch.ones_like(SDF)
rho_nodes[mask] = rho_min

# Integrate element density field
rho_elems = torch.zeros(len(elements))
vol_elems = torch.zeros(len(elements))
for i, (w, xi) in enumerate(zip(model.etype.iweights(), model.etype.ipoints())):
    N = model.etype.N(xi)
    B = model.etype.B(xi)
    J = torch.einsum("jk,mkl->mjl", B, nodes[elements, :])
    detJ = torch.linalg.det(J)
    rho = rho_nodes[elements, None].squeeze() @ N
    rho_elems += w * rho * detJ
    vol_elems += w * detJ

# Normalize
rho_elems /= vol_elems

In [8]:
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
mesh.cell_data["Density"] = rho_elems.numpy()
pl.add_mesh(mesh, cmap="coolwarm", show_edges=True, scalars="Density")
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Compute homogenized properties

In [9]:
# Reduce stiffness with density field
model.material.C *= rho_elems[:, None, None]

# Solve
u, f, σ, ε, α = model.solve()

In [10]:
E_xx = torch.mean(σ[:, 0] / ε[:, 0])
vol_frac = (rho_elems * vol_elems).sum()
print(f"Effective E_xx ({100*vol_frac:.2f}%): {E_xx:.2f}")

Effective E_xx (8.27%): 79.21


In [11]:
# Evaluate displacement
mesh.point_data["Disp"] = u.numpy()

# Contours
contours = mesh.contour([-thickness, thickness], scalars="SDF")

# Plot
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh.outline(), color="k")
pl.add_mesh(contours, cmap="viridis", scalars="Disp")
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…